In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#model.py

import torch.nn as nn
import torch

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):   
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x
             
alexnet = AlexNet()
# alexnet

In [4]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

#建立自己的dataset train_data
class MyDataset(Dataset):
    def __init__(self, image_path: list, image_class: list, transform=None):
        self.image_path = image_path
        self.image_class = image_class
        self.transform = transform

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, item):
        img = Image.open(self.image_path[item]).convert('RGB')
#         if img.mode != 'RGB':
#             raise ValueError("image: {} isn't RGB mode".format(self.image_path[item]))
        label = self.image_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [5]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def get_data(imgs, labels, val_rat=0.2):

    train_imgs_path = []
    train_labels = []
    val_imgs_path = []
    val_labels = []
    supported = ['.jpg', '.JPG']

    # 得到所有图片地址
    images_name = os.listdir(imgs)
    images = [os.path.join(imgs, img) for img in os.listdir(imgs)
              if os.path.splitext(img)[-1] in supported]
    # 得到所有图片label
    df = pd.read_csv(labels)
    path_label_dict = {}
    for i in images_name:
        path_label_dict.update({os.path.join(imgs, i): df[df.image_id == i].iloc[0, 1]})

    # 按比例划分val
    val_num = len(images) * val_rat
    val_path = random.sample(images, int(val_num))

    for img_path in images:
        if img_path in val_path:
            val_imgs_path.append(img_path)
            val_labels.append(path_label_dict[img_path])
        else:
            train_imgs_path.append(img_path)
            train_labels.append(path_label_dict[img_path])

    print("{} images were found in the dataset".format(len(images)))
    print("{} in train_set".format(len(train_labels)))
    print("{} in val_set".format(len(val_labels)))

#     if plot:
#         train_male_num = np.sum(np.array(train_labels) == 1)
#         train_female_num = np.sum(np.array(train_labels) == 0)
#         val_male_num = np.sum(np.array(val_labels) == 1)
#         val_female_num = np.sum(np.array(val_labels) == 0)
#         x = ['train_m', 'train_f', 'val_m', 'val_f']
#         y = [train_male_num, train_female_num, val_male_num, val_female_num]
#         plt.bar(x, y, color=['b', 'r', 'b', 'r'])
#         for a, b, i in zip(x, y, range(len(x))):  # zip 函数
#             plt.text(a, b, "%d" % y[i], ha='center', fontsize=12)  # plt.text 函数
#         plt.rcParams["font.sans-serif"] = ["SimHei"]  # 设置字体
#         plt.title("数据集性别分布")
#         plt.show()

    return train_imgs_path, train_labels, val_imgs_path, val_labels

In [6]:
train_image_path= "/kaggle/input/bitmoji-faces-gender-recognition/BitmojiDataset/trainimages/"
train_lable_path = "/kaggle/input/bitmoji-faces-gender-recognition/train.csv"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

train_imgs, train_labels, val_imgs, val_labels = get_data(train_image_path, train_lable_path, val_rat=0.2)
train_dataset = MyDataset(train_imgs, train_labels, transform)
val_dataset = MyDataset(val_imgs, val_labels, transform)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size)
print(train_loader)
print(val_loader)

3000 images were found in the dataset
2400 in train_set
600 in val_set


In [7]:
import time
import os
import torch
#train and val
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            y+=1
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n


def train(net, train_iter, val_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    loss.to(device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
#             print(X)
#             print(y)
            y+=1
#             print(y)
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        val_acc = evaluate_accuracy(val_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, val acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, val_acc, time.time() - start))
    print("train over!")
    if not os.path.exists('./model_save'): #判断所在目录下是否有该文件名的文件夹
        os.mkdir("./model_save")
    torch.save(net.state_dict(), os.path.join("./model_save/", "epoch_{}_{}.pth".format(epoch+1,1)))
#     torch.save(net, "./model_save/alexnet_epoch5.pth") #保存模型

    
#     torch.save(net.state_dict(), os.path.join("./model_save/", "epoch5.pth")) #保存模型
#     print(alexnet)
    print("model saved!")
    
# model_save_path = "./model_save" 
# alexnet.to(device)
# model = AlexNet()
# model.to(device)
lr, num_epochs = 0.01, 20
# optimizer = torch.optim.Adam(alexnet.parameters(), lr=lr)
optimizer = torch.optim.SGD(alexnet.parameters(), lr=lr) 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train(alexnet, train_loader, val_loader, batch_size, optimizer, device, num_epochs)

# /kaggle/input/bitmoji-faces-gender-recognition/sample_submission.csv

training on  cuda:0
epoch 1, loss 5.1288, train acc 0.482, val acc 0.530, time 35.4 sec
epoch 2, loss 0.8818, train acc 0.474, val acc 0.530, time 16.1 sec
epoch 3, loss 0.7195, train acc 0.520, val acc 0.615, time 16.8 sec
epoch 4, loss 0.7365, train acc 0.512, val acc 0.530, time 16.9 sec
epoch 5, loss 0.7275, train acc 0.533, val acc 0.640, time 16.5 sec
epoch 6, loss 0.6845, train acc 0.585, val acc 0.883, time 16.6 sec
epoch 7, loss 0.7662, train acc 0.617, val acc 0.850, time 16.9 sec
epoch 8, loss 0.5445, train acc 0.748, val acc 0.920, time 16.6 sec
epoch 9, loss 0.4541, train acc 0.864, val acc 0.528, time 16.4 sec
epoch 10, loss 0.4880, train acc 0.867, val acc 0.932, time 16.8 sec
epoch 11, loss 0.2455, train acc 0.922, val acc 0.935, time 16.5 sec
epoch 12, loss 0.7277, train acc 0.858, val acc 0.943, time 16.5 sec
epoch 13, loss 0.2061, train acc 0.932, val acc 0.958, time 16.5 sec
epoch 14, loss 0.1865, train acc 0.944, val acc 0.962, time 16.9 sec
epoch 15, loss 0.1729, 

In [8]:
model_path = '/kaggle/working/model_save/epoch_20_1.pth'
alex_test = AlexNet()
alex_test.load_state_dict(torch.load(model_path))


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

testimages_path = "/kaggle/input/bitmoji-faces-gender-recognition/BitmojiDataset/testimages"
sample_submission_path = "/kaggle/input/bitmoji-faces-gender-recognition/sample_submission.csv"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("test on "+str(device))
alex_test.to(device)
#数据
df = pd.read_csv(sample_submission_path)

alex_test.eval()    #把模型转为test模式
cnt = 0

with torch.no_grad():
    imgs = os.listdir(testimages_path)
    imgs.sort()
    for img_name in imgs:
        img_path = os.path.join(testimages_path, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        img = img.to(device).unsqueeze(0)
        pred = int(alex_test(img).argmax(1))
        df.iloc[cnt, 1] = -1 if pred == 0 else 1
        cnt += 1
#     df = df.iloc[:,[0, 2]]
    df.to_csv("./sample_submission.csv")
    print("test over!")
# df.drop(columns=" ",axis=1)

test on cuda:0
test over!
